In [53]:
import pandas as pd


In [54]:
df = pd.read_csv('/content/drive/MyDrive/AI/DL/fake-news/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [55]:
df = df.dropna()

In [56]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [57]:
X = df.drop('label', axis=1)

In [58]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [59]:
y = df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [60]:
X.shape, y.shape

((18285, 4), (18285,))

In [61]:
import tensorflow as tf


In [62]:
tf.__version__

'2.4.1'

In [63]:
from tensorflow.keras.layers import Embedding

In [64]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [65]:
from tensorflow.keras.models import Sequential

In [66]:
from tensorflow.keras.preprocessing.text import one_hot

In [67]:
from tensorflow.keras.layers import LSTM, Dense

In [68]:
voc_size = 5000

In [69]:
messages = X.copy()

In [70]:
messages.reset_index(inplace=True)

In [71]:
import nltk

In [72]:
import re

In [73]:
from nltk.corpus import stopwords

In [74]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [75]:
from nltk.stem.porter import PorterStemmer

In [76]:
ps = PorterStemmer()

In [77]:
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower().split()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [78]:
one_hot_repr = [one_hot(words,voc_size) for words in corpus]

In [79]:
one_hot_repr

[[4420, 3954, 545, 2081, 2856, 1535, 4054, 3920, 4096, 2420],
 [251, 16, 2706, 3082, 877, 2135, 2732],
 [4484, 1180, 1401, 1506],
 [115, 1333, 2470, 3653, 1003, 554],
 [3061, 877, 324, 4935, 513, 692, 877, 707, 1871, 2259],
 [3423,
  3091,
  4807,
  2316,
  620,
  1233,
  3479,
  1084,
  645,
  2897,
  3403,
  2939,
  1162,
  4318,
  2732],
 [1333, 295, 1374, 513, 2777, 1912, 851, 1593, 4126, 233, 4750],
 [804, 3212, 2710, 4829, 683, 302, 1233, 2453, 4126, 233, 4750],
 [4840, 3620, 847, 679, 4651, 3600, 419, 81, 1233, 4641],
 [4290, 4157, 979, 4695, 2355, 1431, 1191, 110],
 [954, 30, 2950, 2574, 934, 3063, 2786, 547, 1426, 3166, 575],
 [3653, 227, 2856, 3600, 1233, 683],
 [4546, 3443, 4645, 117, 3853, 4540, 779, 1695, 796],
 [2752, 3124, 2879, 4830, 4590, 735, 4455, 4126, 233, 4750],
 [1831, 3837, 1086, 1542, 1843, 4126, 233, 4750],
 [536, 301, 4546, 2490, 1306, 2264, 1966, 4649, 336, 888],
 [4231, 1469, 16],
 [1468, 2337, 1484, 989, 1233, 4941, 2864, 2732],
 [4728, 2301, 2706, 1784, 2

In [80]:
sent_len = 20

In [81]:
embed_doc = pad_sequences(one_hot_repr, padding='pre', maxlen=sent_len)

In [82]:
embed_doc

array([[   0,    0,    0, ..., 3920, 4096, 2420],
       [   0,    0,    0, ...,  877, 2135, 2732],
       [   0,    0,    0, ..., 1180, 1401, 1506],
       ...,
       [   0,    0,    0, ..., 4126,  233, 4750],
       [   0,    0,    0, ..., 1839, 3810, 2680],
       [   0,    0,    0, ..., 1501, 3475, 3604]], dtype=int32)

In [83]:
embed_vect_feat = 40

In [84]:
model = Sequential()

In [85]:
model.add(Embedding(voc_size, embed_vect_feat,input_length=sent_len))

In [86]:
from tensorflow.keras.layers import Dropout

In [87]:
model.add(Dropout(0.3))

In [88]:
model.add(LSTM(100))

In [89]:
model.add(Dropout(0.3))

In [90]:
model.add(Dense(1,activation='sigmoid'))

In [91]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [92]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [93]:
import numpy as np


In [94]:
X = np.array(embed_doc)

In [95]:
y = np.array(y)

In [96]:
from sklearn.model_selection import train_test_split

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [98]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 11ms/step - loss: 0.4934 - accuracy: 0.7290 - val_loss: 0.2066 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.1561 - accuracy: 0.9406 - val_loss: 0.1973 - val_accuracy: 0.9175
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.1071 - accuracy: 0.9610 - val_loss: 0.2122 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0812 - accuracy: 0.9700 - val_loss: 0.2520 - val_accuracy: 0.9186
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0627 - accuracy: 0.9772 - val_loss: 0.2545 - val_accuracy: 0.9143
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0615 - accuracy: 0.9781 - val_loss: 0.2940 - val_accuracy: 0.9142
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0453 - accuracy: 0.9857 - val_loss: 0.3848 - val_accuracy: 0.9135
Epoch

In [99]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [100]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [101]:
confusion_matrix(y_pred,y_test)

array([[3148,  294],
       [ 271, 2322]])

In [102]:
accuracy_score(y_pred,y_test)

0.9063794531897266